# Primitives — Extra Figures

Complementary visuals to the main primitives notebook: 2D diffusion heatmap and a rolling path dispersion animation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import jax, jax.numpy as jnp


## 2D Diffusion Heatmap (Toy)
State-dependent diagonal diffusion magnitude visualized over a grid.

In [ ]:
g = np.linspace(-2.0, 2.0, 80)
X1, X2 = np.meshgrid(g, g)
sig = lambda u: 1/(1+np.exp(-u))
s1 = 0.1 + 0.2*sig(X1)
s2 = 0.1 + 0.2*sig(X2)
Sigma_norm = np.sqrt(s1**2 + s2**2)
figH, axH = plt.subplots(figsize=(4.8,3.6))
im = axH.imshow(Sigma_norm, extent=[g.min(), g.max(), g.min(), g.max()], origin='lower', cmap='viridis', aspect='auto')
figH.colorbar(im, ax=axH); axH.set_title('||sigma(x)||_F over grid'); axH.set_xlabel('x1'); axH.set_ylabel('x2')
figH.tight_layout(); figH


## Rolling Path Dispersion (Animation)
Animate how the cross-sectional dispersion of a toy OU process evolves across paths.

In [ ]:
# Toy OU simulation (noise only)
steps, paths, dt = 120, 64, 0.02
x = np.zeros((paths,))
xs = [x.copy()]
for i in range(steps):
    dW = np.random.normal(0, np.sqrt(dt), size=(paths,))
    x = 0.98*x + 0.3*dW
    xs.append(x.copy())
xs = np.stack(xs, axis=0)
fig, ax = plt.subplots(figsize=(5,3))
ax.set_title('Cross-sectional dispersion over time'); ax.set_xlabel('time step'); ax.set_ylabel('std across paths')
line, = ax.plot([], [], lw=2)
def init():
    line.set_data([], [])
    return (line,)
def animate(i):
    t = np.arange(i+1)
    s = xs[:i+1].std(axis=1)
    line.set_data(t, s)
    ax.set_xlim(0, steps); ax.set_ylim(0, xs.std()*1.2 + 1e-6)
    return (line,)
ani = animation.FuncAnimation(fig, animate, init_func=init, frames=steps, interval=100, blit=True)
ani.to_jshtml()
